# SEIRHVD Data Fit - Multiple Scenarios
This Jupyter notebooks implements the SEIRHVD4 model for fitting the actual Chilean data in order to make projections in bed usage, amount of infected and deaths. 
This implements multilpe instances at at time to facilitate fitting the data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

Linux


## Time Variables

In [4]:
initdate = datetime(2020,5,15)
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data

In [5]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importando Datos
Import Population
Infectados Activos
Infectados Acumulados
Infectados diarios
Sochimi
Fallecidos Acumulados
Infectados Activos Minciencia
Done


# Simulation in multiple betas

## Simulation Parameters

In [6]:
# Simulation parameters
tsim = 1000
beta = [0.1,0.12,0.14,0.16]
mu = 1.5
k = 0

SeroPrevFactor=1
expinfection=0

## Quarantines

In [7]:
# Quarantines 
max_mob = 0.8
rem_mob = 0.5

#alpha = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once').alpha(t)
alpha = Quarantine(rem_mob).alpha

## Simulation
Run parallel simulations on different beta's

In [17]:
simulation = []
for i in beta:
    aux = SEIRHVD(tsim,i,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM)
    aux.integr_sci(0,tsim,0.1)
    simulation.append(aux)

InitialCondition Object Data
InitialCondition Object Data
InitialCondition Object Data
InitialCondition Object Data


# Analysis



In [13]:
days = currentday+20
index = [np.searchsorted(simulation[i].t,days) for i in range(len(beta))]

## Active Infected

In [16]:
tr_index = np.searchsorted(RM.tr,days)
plt.xlim(0,days)
plt.scatter(RM.tr,RM.Ir,label='Real Data')
for i in range(len(beta)):
    plt.plot(simulation[i].t[:index[i]],simulation[i].I[:index[i]],label='beta = '+str(beta[i]))

plt.legend(loc=0)
plt.title('Active Infected')
plt.show()


## Daily Infected

In [38]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d[:index],label='Total Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Acummulated Infected

In [16]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Deaths

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.B[:index],label='Total Acummulated Deaths')
plt.scatter(RM.Br_tr,RM.Br,label='Real Data')
plt.legend(loc=0)
plt.show()

## UCI/UTI

In [40]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.Hse[:index]+simulation.Hout[:index],label='UCI/UTI Beds')
plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.legend(loc=0)
plt.show()

## VMI

In [20]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.V[:index],label='VMI Usage')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.legend(loc=0)
plt.show()

# Error

## Error a lo largo de la curva

In [47]:
t1_date = datetime(2020,8,15)
t1_day = (t1_date-initdate).days
t1_idx = np.where(np.array(RM.tr)>=t1_day)[0][0]

In [57]:
idx = np.searchsorted(simulation.t,RM.tr[:t1_idx])
Err = np.sum(abs(RM.Ir[:t1_idx]-simulation.I[idx]))/np.mean(RM.Ir[:t1_idx])

Err_norm = LA.norm(RM.Ir[:t1_idx]-simulation.I[idx])/np.mean(RM.Ir[:t1_idx])

In [58]:
Err

528.5024695326207

In [59]:
Err_norm

134.78450020801006

## Error total infectados